In [77]:
import os, glob, json

path = "/Users/[editted]/Desktop/compSoc/datasets/indeed_scraped_data/job_url_data"

# print (len(path)) #77

d = {}

for filename in glob.glob(os.path.join(path, '*.csv')): #for each file in job_url_data
    date = filename[100:107] #77 + 23, the index where the date starts in each file
    with open(filename, "r") as f:
        for line in f.readlines():
            line = line[:len(line)-1] #gets rid of /n
            if line in d:
                if date < d[line]: #replaces with smallest date
                    d[line] = str(date)
            elif line not in d and ".com" in line: d[line] = str(date) #else adds to dictionary, gets rid of extra row
    
os.chdir("/Users/[editted]/Desktop/compSoc/datasets/indeed_scraped_data")

json_obj = json.dumps(d, indent = 4)

with open("jobs.json", "w") as outFile:
    outFile.write(json_obj)
    
print (len(d)) #21260 unique job urls

21260


There are 21260 unique job urls in our database, collected between 5/17 and 5/23.

In [80]:
import pandas as pd
import os
import json

os.chdir("/Users/[editted]/Desktop/compSoc/datasets/indeed_scraped_data/job_info_data")
path = "/Users/[editted]/Desktop/compSoc/datasets/indeed_scraped_data/job_info_data"

fol = os.listdir(path)
dfs = []
                
for f in fol:
    if '.csv' in f:
        df = pd.read_csv(f)
        df.rename(columns = {"lnks_link": "link", "lnks_job_title": "job_title", 
                             "lnks_job_description": "job_description", "lnks_company": "company",
                             "lnks_company_url": "company_url", "lnks_company_location": 
                             "company_location"}, inplace = True)
        dfs.append(df)
        
    elif '.json' in f: #for json files only
        with open(f) as file:
            df_list = []
            data = json.load(file)
            for i in data: #keep all entries with all fields filled out
                for item in data[i]:
                    if len(item) == 6:
                        df_list.append(item)
        df = pd.DataFrame(df_list) #convert to pandas dataframe
        dfs.append(df)

indeed_df = pd.concat(dfs)
indeed_df.dropna(subset = ["job_title", "job_description"], inplace = True)
indeed_df.drop_duplicates(subset = ["link"], inplace = True)
print(indeed_df.shape)

os.chdir("/Users/[editted]/Desktop/compSoc/datasets/indeed_scraped_data")
indeed_df.to_csv("clean_indeed.csv", index = False)

(15987, 6)


There are 15987 unique job offerings from 5/17 to 5/23.

In [84]:
import os
import pandas as pd
import json

os.chdir("/Users/[editted]/Desktop/compSoc/datasets/indeed_scraped_data")
path = "/Users/[editted]/Desktop/compSoc/datasets/indeed_scraped_data"

with open("jobs.json") as file:
    df_list = []
    data = json.load(file)
    url_list = data.items()
    for i in url_list:
        df_list.append(i)
    url_df = pd.DataFrame(df_list)
url_df.rename(columns = {0: "link", 1: "date"}, inplace = True)

clean_df = pd.read_csv("clean_indeed.csv")
clean_df

df = url_df.merge(clean_df, how = "left", on="link")

for col in df.columns:
    print(col, ' not null count:', df[col].notnull().sum()) #5444 not matched, 1666 had null companies/urls originally

df["match_ind"] = None
df["match_ind"][df["job_title"].isna()] = "unmatched"
df["match_ind"][df["job_title"].notnull()] = "matched"

df2 = df.groupby(["match_ind", "date"]).agg({'link': 'count'}).reset_index()
df2 #most of the unmatched are on the last day, there are much fewer job offerings available in later days

df3 = df.dropna().groupby(["date"]).agg({'link': 'count'}).reset_index()
df3

link  not null count: 21260
date  not null count: 21260
job_title  not null count: 15816
company  not null count: 14153
company_url  not null count: 14153
company_location  not null count: 15816
job_description  not null count: 15816


,date,link
0,5172022,3225
1,5182022,1220
2,5192022,3343
3,5202022,4123
4,5212022,1726
5,5222022,516


Percent matched = #not null/#null = 66.5%

Missing data are primarily on 5/17 and 5/23.

Matched data varies greatly each day, with the highest frequency occurring on 5/20/2022.

Complete job listings also follow the same pattern as the matched data, varying greatly each day and decreasing sharply after the peak.

I would not consider these results to be particularly bad with respect to data quality; however, I would need to see data from other weeks to confirm that this was due to random noise and not a consistent trend. If it is a trend, this could pose a problem—the data we collect in later days of the week is more likely to be incomplete, which means our analysis will more likely focus on data from specific days of the week. This could introduce bias; for example, if these jobs are more likely to be posted on weekend days, our analysis would now not capture that effect.